# New Raster Mask

Created 2019-01-15
This program takes the code from Raster Mask and cleans it up for use to do two things:

1. Create new rasters with a simple mask applied
2. Complete connected components of two masks

Check crs of raster
'EPSG:4326' ... or 

In [25]:
import rasterio
from rasterio.mask import mask
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

### Inputs

In [20]:
temp_data = '../../temp_data/'
raster_interim = '../../data/interim/'
raster_raw = '../../data/raw/'

### Mask Functions

In [24]:
# Let's try to make the mask directy with rasterio

def raster_mask_poly(raster, band, density, crs):
    """Fucntion makes numpy array for of raster then applies mask.
    requires rasterio object, band number, and density as float on int and crs of raster
    returns dict of polygons and numpy mask
    """

    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros 
    mask[mask >= density] = 1 
    
    # Extract feature shapes and values from the array.
    for geom, val in rasterio.features.shapes(mask, transform=raster.transform):
        
        # Transform shapes from the dataset's own coordinate
        # reference system to CRS84 (EPSG:4326) removed 2019-01-15
        geom = rasterio.warp.transform_geom(raster.crs, crs, geom, precision=6) # WGS84 (EPSG 4326)
    
    # turn geom into Polygon object from shapely    
    # Polygon(geom['coordinates'][0])
    poly_gdf = gpd.GeoDataFrame()
    for i, poly in enumerate(geom['coordinates']):
        poly_gdf.loc[i,'geometry']= Polygon(poly)

    return mask, geom, poly_gdf

In [13]:
def raster_mask(raster, band, density):
    "Function returns raster mask"
    
    mask = raster.read(band)
    mask[mask < density] = 0 # mask as zeros and ones
    mask[mask >= density] = 1
    
    return mask

### Polygons for >1500 ppl per km2

In [34]:
file = rasterio.open(raster_interim+'LandScan2015_w001001_Clip.tif')
crs = file.crs

In [35]:
mask1500m, geom1500, poly_gdf1500 = raster_mask_poly(file, 1, 1500, crs)

In [37]:
print(poly_gdf1500.head(6))
print(len(poly_gdf1500))

                                            geometry
0  POLYGON ((-25.466667 37.441667, -25.466667 -34...
1  POLYGON ((20.258333 -34.566667, 20.266667 -34....
2  POLYGON ((9.824999999999999 37.3, 9.8249999999...
3  POLYGON ((20.225 -34.558333, 20.233333 -34.558...
4  POLYGON ((9.85 37.3, 9.85 37.283333, 9.8416669...
5  POLYGON ((22.033333 -34.191667, 22.033333 -34....
36174


In [38]:
# drop first row, which is bounding box of Africa and the write 
poly_gdf1500 = poly_gdf1500.drop(poly_gdf1500.index[0], axis = 0)
poly_gdf1500.head(6)

,geometry
1,"POLYGON ((20.258333 -34.566667, 20.266667 -34...."
2,"POLYGON ((9.824999999999999 37.3, 9.8249999999..."
3,"POLYGON ((20.225 -34.558333, 20.233333 -34.558..."
4,"POLYGON ((9.85 37.3, 9.85 37.283333, 9.8416669..."
5,"POLYGON ((22.033333 -34.191667, 22.033333 -34...."
6,"POLYGON ((19.225 -34.416667, 19.233333 -34.416..."


In [39]:
# write out polygons 

poly_gdf1500.to_file(temp_data+'LandScan2015_w001001_Clip.tif_1500.shp', driver='ESRI Shapefile')